Importing needed packages:

In [31]:
import pandas as pd

Reading original data sheets:

In [32]:
df1 = pd.read_excel('data.xlsx', sheet_name='REPORT_2019_CLEARED')
df2 = pd.read_excel('data.xlsx', sheet_name='REPORT_2020_CLEARED')
df3 = pd.read_excel('data.xlsx', sheet_name='REPORT_2021')
df4 = pd.read_excel('data.xlsx', sheet_name='REPORT_2022_cleared')

dfs = [df1, df2, df3, df4]
new = []

years = [2019, 2020, 2021, 2022]

Initialising function for processing our data sheets.

In [33]:
def process_dataframes(dfs, years):
    new = []
    
    # transposing and cleaning up each dataframe
    for i in range(len(dfs)):
        dfs[i].rename(columns={'Unnamed: 0': 'index'}, inplace=True)
        
        transposed_df = dfs[i].T
        
        transposed_df.columns = transposed_df.iloc[0]
        
        transposed_df = transposed_df.iloc[1:].reset_index()
        
        # renaming the 'index' column to 'City'
        transposed_df.rename(columns={'index': 'City'}, inplace=True)
        
        # handling duplicated columns
        if transposed_df.columns.duplicated().any():
            transposed_df = transposed_df.loc[:, ~transposed_df.columns.duplicated()]
        
        new.append(transposed_df)

    # adding 'Year' column and concatenating all dataframes
    final_df_list = []
    for i in range(len(years)):
        new[i]['Year'] = years[i]
        final_df_list.append(new[i].reset_index(drop=True))
    
    # concatenating all dataframes
    df = pd.concat(final_df_list, ignore_index=True)
    
    # droping columns where all values are NaN
    df.dropna(how='all', axis=1, inplace=True)
    
    # reordering columns to put 'Year' after 'City'
    cols = df.columns.tolist()
    cols.remove('Year')
    city_index = cols.index('City') + 1
    cols.insert(city_index, 'Year')
    df = df[cols]
    
    return df

Calling that function for original data:

In [34]:
processed_original_data = process_dataframes(dfs,years)
processed_original_data.to_excel('processed_original_data.xlsx', index=False)

After that we faced a trouble of having identical columns in each year but having slightly different namings. 

We decided to use SequenceMatcher in order to get a list of columns that have similarity more than 90%.

Initialising function to check similarities in columns names:

In [35]:
from difflib import SequenceMatcher

def find_similar_columns(df, threshold, output_file):
    similar_columns = {}
    columns = df.columns.tolist()
    
    # comparing each column with every other column
    for i, col1 in enumerate(columns):
        for col2 in columns[i + 1:]:
            seq_match = SequenceMatcher(None, col1, col2)
            ratio = seq_match.ratio()
            
            # if similarity ratio exceeds threshold, add to the dictionary
            if ratio > threshold:
                if col1 not in similar_columns:
                    similar_columns[col1] = []
                similar_columns[col1].append(col2)
    
    data = []
    for col, similar in similar_columns.items():
        row = [col] + similar
        data.append(row)
    
    # finding maximum number of similar columns for formatting
    max_similar = max(len(row) for row in data)
    
    columns = ['Column'] + [f'Similar Column {i+1}' for i in range(max_similar - 1)]
    
    similar_columns_df = pd.DataFrame(data, columns=columns)
    
    similar_columns_df.to_excel(output_file, index=False)
    
    print(f"Data saved to '{output_file}'")

Calling that function for original processed data:

In [36]:
find_similar_columns(processed_original_data,0.9,'similar_columns_original.xlsx')

Data saved to 'similar_columns_original.xlsx'


Results are close but not enough. 

There are columns that have only 1 different word, what technically makes them pass the threshold. However, that 1 word is enough for columns to represent absolutely unalike data. 

We decided to try another approach: manual 'cleaning' of original dataset. That fixes next issues:
* removing columns that have no informational value
* unifying namings

For the code we are leaving previously implemented things: deleting empty columns, transposing and joining sheets together.

In [37]:
df1_manual = pd.read_excel('manual_data.xlsx', sheet_name='REPORT_2019_CLEARED')
df2_manual = pd.read_excel('manual_data.xlsx', sheet_name='REPORT_2020_CLEARED')
df3_manual = pd.read_excel('manual_data.xlsx', sheet_name='REPORT_2021')
df4_manual = pd.read_excel('manual_data.xlsx', sheet_name='REPORT_2022_cleared')

dfs_manual = [df1_manual, df2_manual, df3_manual, df4_manual]
new = []

In [38]:
processed_manual_data = process_dataframes(dfs_manual, years)
processed_manual_data.to_excel('processed_manual_data.xlsx', index=False)

In [39]:
find_similar_columns(processed_manual_data, 0.9, 'similar_columns_manual.xlsx')

Data saved to 'similar_columns_manual.xlsx'


Results are much better and looks like the best possible output to work with.

Another approach - LLM. First step is preparing minimally data to feed it to model. 

In [40]:
# #FOR GPT
# 
# for i in range(4):
#     # transposing matrix
#     dfs[i].rename(columns={'Unnamed: 0': 'index'}, inplace=True)
#     new.append(dfs[i].T)
# 
#     # Вибираємо перший запис і робимо його колонками
#     new[i].columns = new[i].iloc[0]
# 
#     # Вибираємо усі записи окрім першого
#     new[i] = new[i].iloc[1:]
# 
#     # Оновлюємо індекс
#     new[i] = new[i].reset_index()
# 
#     # Перейменовуємо індекс на City
#     new[i].rename(columns={'index': 'City'}, inplace=True)
#     if True in new[i].columns.duplicated():
#         new[i] = new[i].loc[:, ~new[i].columns.duplicated()]
#    
# years = ['2019', '2020', '2021', '2022']
# f = []
# for i in range(4):
#     new[i]['Year'] = years[i]
#     f.append(new[i].reset_index(drop=True))
# 
# df = pd.concat(f, ignore_index=True)
# 
# cols = df.columns.tolist()
# 
# # Removing the "Year" column from its current position
# cols.remove('Year')
# 
# # Inserting the "Year" column back right after "City"
# city_index = cols.index('City') + 1
# cols.insert(city_index, 'Year')
# 
# # Reordering
# df = df[cols]
# 
# df.to_excel('data_for_gpt.xlsx', index=False)